In [2]:
# Imports
from bs4 import BeautifulSoup
import requests, json
import pandas as pd

In [29]:
default_parameters = {
    'search_query':'Data Science',
    'location':'20001',
    'miles':15,
    'ordered_keywords':['Investment','Banking','Finance','Hedge','Python','Fintech','SQL','Analysis','Modelling'],
    'exclude_keywords':['Recruitment','Headhunter','Manager','Director','Senior'],
    'title_keywords':['Entery','Junior'],
    'pages':10
}

In [30]:
def create_url(parameters):
    # create base url for all further searches
    what = parameters['search_query'].replace(" ","+")
    where = parameters['location'].replace(" ","+")
    miles = parameters['miles']
    base_url = f"https://www.indeed.co.uk/jobs?q={what}&l={where}&radius={miles}"
    return base_url

In [22]:
def rate_job(j_title, j_soup, parameters):
    # rate job by keywords
    description = j_soup.find(id="jobDescriptionText").get_text()
    keywords = parameters['ordered_keywords']
    title_keywords = parameters['title_keywords']
    exclude_keywords = parameters['exclude_keywords']
    total_keywords = len(keywords) + len(title_keywords)
    keywords_present = []
    title_keywords_present = []
    rating = 0
    
    # Check for keyword, add value to rating depending on ranking
    for index,keyword in enumerate(keywords):
        if keyword in description:
            rating += len(keywords) - index
            keywords_present.append(keyword)
    
    # Check for title keywords
    for index,keyword in enumerate(title_keywords):
        if keyword in j_title:
            rating += total_keywords - index
            title_keywords_present.append(keyword)
    
    # Normalise rating
    rating = rating/sum(range(1,total_keywords+1))
    
    # Check for excluded keywords
    for keyword in exclude_keywords:
        if keyword in j_title:
            rating = 0
            break
    
    return description,rating,keywords_present,title_keywords_present


In [31]:
def get_job_details(job,parameters):
    
    # Get link and title
    job_url = job.find(class_='title').a['href']
    
    # Correct for truncated URLs
    job_url = "https://www.indeed.co.uk" + job_url if (job_url.startswith("/")) else job_url
    job_page = requests.get(job_url)
    job_soup = BeautifulSoup(job_page.content,'html.parser')
    
    # Give URL after redirect (ads/analytics etc.)
    job_url = job_page.url 
    
    # Get job title and company name
    title = job.find(class_='title').a['title']
    company = job_soup.find(class_="icl-u-lg-mr--sm").get_text()
    
    # Get description, rating and present keywords
    description, rating, keywords_present, title_keywords_present = rate_job(title,job_soup,parameters)
    
    return title, company, job_url, description, rating, keywords_present, title_keywords_present

In [32]:
def scrape(parameters):
    
    # Create base url for all further searches
    base_url = create_url(parameters)
    
    # Output list and frame
    output = []
    
    for x in range(0,parameters['pages']):
        if(x==0):
            page_append = ""
        else: 
            page_append = "&start=" + str(x*10)
            
        # get page
        current_page = requests.get(base_url+page_append,timeout=5)
        page_soup = BeautifulSoup(current_page.content,"html.parser")
        
        for job in page_soup.select(".jobsearch-SerpJobCard"):
            title, company, url, description, rating, keywords_present, title_keywords_present = get_job_details(job,parameters)
            output.append([rating,title,company,description,url,keywords_present,title_keywords_present,x+1])
            
        print(f"Page {x+1} completed",end="\r")
        
    df_output_frame = pd.DataFrame(
        output,columns=['Rating','Job Title','Company','Description','Job URL','Keywords Present','Title Keywords','Page Found']).sort_values(
        by='Rating',ascending=False).reset_index(drop=True)

    return df_output_frame

In [33]:
jobs = scrape(default_parameters)

In [34]:
len(jobs)

0